# 01. Bronze Layer: ダミーデータ生成

## 概要
Paper08（関節リウマチの有病率と治療動向）の再現に必要なダミーデータを生成します。

## 生成するデータ
1. **レセプト基本データ (RE)** - 患者基本情報
2. **傷病データ (SY)** - 診断コード（ICD-10）
3. **医薬品データ (IY)** - 処方薬情報
4. **診療行為データ (SI)** - 手術・検査情報

## データ設計方針
- 論文のDefinition 3に基づくRA患者を含むように設計
- 年齢分布は論文のTable 2に近似
- 薬剤使用パターンは論文のTable 3に近似

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import hashlib
import os

np.random.seed(42)

# 出力先ディレクトリ
BRONZE_DIR = "../data/bronze"
os.makedirs(BRONZE_DIR, exist_ok=True)

print("ダミーデータ生成を開始します...")

ダミーデータ生成を開始します...


## 1. 定数・マスタデータの定義

In [ ]:
# 論文の設定に基づく定数
STUDY_PERIOD_START = "2017-04-01"
STUDY_PERIOD_END = "2018-03-31"
REFERENCE_DATE = "2017-10-01"  # 人口統計の基準日

# サンプルサイズ（論文の1/1000スケール）
N_TOTAL_PATIENTS = 10000  # 論文では約100万人
RA_PREVALENCE = 0.0065    # 論文のDefinition 3での有病率

# 年齢層定義（論文Table 2に基づく）
AGE_GROUPS = [
    ("16-19", 16, 19),
    ("20-29", 20, 29),
    ("30-39", 30, 39),
    ("40-49", 40, 49),
    ("50-59", 50, 59),
    ("60-69", 60, 69),
    ("70-79", 70, 79),
    ("80-84", 80, 84),
    ("85+", 85, 100)
]

# 論文Table 2に基づく年齢層別RA患者割合（%）
AGE_GROUP_RA_DISTRIBUTION = {
    "16-19": 0.5,
    "20-29": 1.5,
    "30-39": 4.8,
    "40-49": 10.1,
    "50-59": 14.9,
    "60-69": 26.4,
    "70-79": 28.6,
    "80-84": 6.1,
    "85+": 7.0
}

# RA関連ICD-10コード
RA_ICD10_CODES = [
    "M050", "M051", "M052", "M053", "M058", "M059",  # 血清陽性RA
    "M060", "M062", "M063", "M068", "M069",          # その他のRA（M061, M064除く）
    "M080", "M083", "M084", "M088", "M089"           # 若年性関節炎（M081, M082除く）
]

# 非RA関連ICD-10コード（コントロール用）
NON_RA_ICD10_CODES = [
    "J00", "J06", "I10", "E11", "K21", "M54", "G43", "F32"
]

print(f"総患者数: {N_TOTAL_PATIENTS}")
print(f"想定RA患者数: {int(N_TOTAL_PATIENTS * RA_PREVALENCE)}")

## 2. RA治療薬マスタの定義

In [ ]:
# DMARDsの医薬品コード（簡略化したダミーコード）
# 実際のNDBでは厚労省の医薬品マスタに基づくコード

DMARDS_MASTER = {
    # csDMARDs（従来型合成DMARD）
    "MTX": {"codes": ["1199101", "1199102"], "name": "メトトレキサート", "category": "csDMARD"},
    "SSZ": {"codes": ["1199201"], "name": "サラゾスルファピリジン", "category": "csDMARD"},
    "TAC": {"codes": ["1199301"], "name": "タクロリムス", "category": "csDMARD"},
    "BUC": {"codes": ["1199401"], "name": "ブシラミン", "category": "csDMARD"},
    "IGT": {"codes": ["1199501"], "name": "イグラチモド", "category": "csDMARD"},
    "LEF": {"codes": ["1199601"], "name": "レフルノミド", "category": "csDMARD"},
    
    # bDMARDs - TNF阻害薬
    "IFX": {"codes": ["4400101"], "name": "インフリキシマブ", "category": "TNFI"},
    "ETN": {"codes": ["4400102"], "name": "エタネルセプト", "category": "TNFI"},
    "ADA": {"codes": ["4400103"], "name": "アダリムマブ", "category": "TNFI"},
    "GLM": {"codes": ["4400104"], "name": "ゴリムマブ", "category": "TNFI"},
    "CZP": {"codes": ["4400105"], "name": "セルトリズマブペゴル", "category": "TNFI"},
    
    # bDMARDs - IL-6阻害薬
    "TCZ": {"codes": ["4400201"], "name": "トシリズマブ", "category": "IL6I"},
    "SAR": {"codes": ["4400202"], "name": "サリルマブ", "category": "IL6I"},
    
    # bDMARDs - アバタセプト
    "ABT": {"codes": ["4400301"], "name": "アバタセプト", "category": "ABT"},
    
    # tsDMARDs - JAK阻害薬
    "TOF": {"codes": ["4400401"], "name": "トファシチニブ", "category": "JAKi"},
    "BAR": {"codes": ["4400402"], "name": "バリシチニブ", "category": "JAKi"},
}

# 経口ステロイド
CS_CODES = ["2454001", "2454002", "2454003"]  # プレドニゾロン等

# NSAIDs
NSAID_CODES = ["1141001", "1141002", "1141003"]

# 全DMARDsコードのリスト
ALL_DMARD_CODES = [code for drug in DMARDS_MASTER.values() for code in drug["codes"]]

print(f"DMARDs種類数: {len(DMARDS_MASTER)}")
print(f"DMARDsコード数: {len(ALL_DMARD_CODES)}")

## 3. 患者基本情報の生成

In [ ]:
def generate_hash_key(patient_id: int) -> str:
    """共通キーを生成（実際のNDBでは保険証情報等からハッシュ化）"""
    return hashlib.sha256(f"patient_{patient_id}".encode()).hexdigest()[:32]

def generate_birth_date(age: int, reference_date: str) -> str:
    """年齢から生年月日を逆算"""
    ref = datetime.strptime(reference_date, "%Y-%m-%d")
    birth_year = ref.year - age
    birth_month = np.random.randint(1, 13)
    birth_day = np.random.randint(1, 29)
    return f"{birth_year}-{birth_month:02d}-{birth_day:02d}"

def assign_age_group(age: int) -> str:
    """年齢から年齢群を決定"""
    for group_name, min_age, max_age in AGE_GROUPS:
        if min_age <= age <= max_age:
            return group_name
    return "Unknown"

# 患者データの生成
n_ra_patients = int(N_TOTAL_PATIENTS * RA_PREVALENCE * 1.5)  # Definition 0相当（後でフィルタ）
n_non_ra_patients = N_TOTAL_PATIENTS - n_ra_patients

patients = []

# RA患者の生成（年齢分布を論文に合わせる）
for i in range(n_ra_patients):
    # 年齢群を論文の分布に従って選択
    age_group = np.random.choice(
        list(AGE_GROUP_RA_DISTRIBUTION.keys()),
        p=[v/100 for v in AGE_GROUP_RA_DISTRIBUTION.values()]
    )
    
    # 年齢群から具体的な年齢を決定
    for group_name, min_age, max_age in AGE_GROUPS:
        if group_name == age_group:
            age = np.random.randint(min_age, max_age + 1)
            break
    
    # 性別（論文では女性76.3%）
    sex = "2" if np.random.random() < 0.763 else "1"  # 1:男性, 2:女性
    
    patients.append({
        "patient_id": i,
        "共通キー": generate_hash_key(i),
        "age": age,
        "age_group": age_group,
        "sex": sex,
        "birth_date": generate_birth_date(age, REFERENCE_DATE),
        "is_ra_candidate": True
    })

# 非RA患者の生成（一般人口の年齢分布）
for i in range(n_non_ra_patients):
    patient_id = n_ra_patients + i
    age = np.random.randint(16, 90)
    sex = "2" if np.random.random() < 0.51 else "1"
    
    patients.append({
        "patient_id": patient_id,
        "共通キー": generate_hash_key(patient_id),
        "age": age,
        "age_group": assign_age_group(age),
        "sex": sex,
        "birth_date": generate_birth_date(age, REFERENCE_DATE),
        "is_ra_candidate": False
    })

df_patients = pd.DataFrame(patients)
print(f"生成した患者数: {len(df_patients)}")
print(f"  - RA候補患者: {df_patients['is_ra_candidate'].sum()}")
print(f"  - 非RA患者: {(~df_patients['is_ra_candidate']).sum()}")
print(f"\n年齢群分布（RA候補）:")
print(df_patients[df_patients['is_ra_candidate']]['age_group'].value_counts())

## 4. レセプト基本データ (RE) の生成

In [ ]:
def generate_receipt_records(df_patients: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    """
    各患者に対して月次のレセプトレコードを生成
    """
    records = []
    
    # 診療月の生成（2017年4月〜2018年3月）
    months = pd.date_range(start=start_date, end=end_date, freq='MS')
    
    for _, patient in df_patients.iterrows():
        # 受診月数を決定（RA候補は多め）
        if patient['is_ra_candidate']:
            n_visits = np.random.randint(2, 13)  # 2-12ヶ月
        else:
            n_visits = np.random.randint(1, 5)   # 1-4ヶ月
        
        visit_months = np.random.choice(months, size=min(n_visits, len(months)), replace=False)
        
        for visit_month in visit_months:
            records.append({
                "共通キー": patient["共通キー"],
                "検索番号": f"RCP{patient['patient_id']:08d}{visit_month.strftime('%Y%m')}",
                "データ識別": "1",  # 医科
                "行番号": "1",
                "枝番号": "0",
                "レコード識別情報": "RE",
                "レセプト番号": f"{np.random.randint(1000000, 9999999)}",
                "レセプト種別": "1112",  # 医科入院外
                "診療年月": visit_month.strftime("%Y%m"),
                "男女区分": patient["sex"],
                "生年月日": patient["birth_date"],
                "fy": "2017" if visit_month.month >= 4 else "2017",
                "year": visit_month.strftime("%Y"),
                "month": visit_month.strftime("%m"),
                "prefecture_number": f"{np.random.randint(1, 48):02d}"
            })
    
    return pd.DataFrame(records)

df_re = generate_receipt_records(df_patients, STUDY_PERIOD_START, STUDY_PERIOD_END)
print(f"生成したレセプトレコード数: {len(df_re)}")
print(f"\n診療年月の分布:")
print(df_re['診療年月'].value_counts().sort_index())

## 5. 傷病データ (SY) の生成

In [ ]:
def generate_disease_records(df_re: pd.DataFrame, df_patients: pd.DataFrame) -> pd.DataFrame:
    """
    各レセプトに対して傷病名レコードを生成
    RA候補患者にはRA関連ICD-10コードを付与
    """
    records = []
    
    # 患者の属性をマージ
    df_merged = df_re.merge(
        df_patients[['共通キー', 'is_ra_candidate']],
        on='共通キー',
        how='left'
    )
    
    for _, receipt in df_merged.iterrows():
        # 傷病名数を決定
        n_diseases = np.random.randint(1, 5)
        
        for j in range(n_diseases):
            if receipt['is_ra_candidate'] and j == 0:
                # RA候補患者の主病名はRA関連コード
                icd10 = np.random.choice(RA_ICD10_CODES)
            else:
                # その他は非RA関連コード
                icd10 = np.random.choice(NON_RA_ICD10_CODES)
            
            # 診療開始日（診療年月の1日〜28日）
            year_month = receipt['診療年月']
            start_day = np.random.randint(1, 29)
            
            records.append({
                "共通キー": receipt["共通キー"],
                "検索番号": receipt["検索番号"],
                "データ識別": receipt["データ識別"],
                "行番号": str(j + 1),
                "枝番号": "0",
                "レコード識別情報": "SY",
                "傷病名コード": f"SY{icd10}",
                "診療開始日": f"{year_month}{start_day:02d}",
                "転帰区分": np.random.choice(["1", "2", "3"]),  # 1:継続, 2:治癒, 3:中止
                "修飾語コード": "",
                "傷病名": f"傷病名_{icd10}",
                "ICD10コード": icd10,
                "fy": receipt["fy"],
                "year": receipt["year"],
                "month": receipt["month"],
                "prefecture_number": receipt["prefecture_number"]
            })
    
    return pd.DataFrame(records)

df_sy = generate_disease_records(df_re, df_patients)
print(f"生成した傷病レコード数: {len(df_sy)}")
print(f"\nICD-10コードの分布（上位10）:")
print(df_sy['ICD10コード'].value_counts().head(10))

## 6. 医薬品データ (IY) の生成

In [ ]:
# 論文Table 3に基づく薬剤使用率（全年齢）
DRUG_USAGE_RATES = {
    "MTX": 0.634,    # 63.4%
    "SSZ": 0.249,    # 24.9%
    "BUC": 0.145,    # 14.5%
    "TAC": 0.106,    # 10.6%
    "IGT": 0.047,    # 4.7%
    "LEF": 0.030,    # 3.0%
    "TNFI": 0.135,   # 13.5% (bDMARDs全体の約60%)
    "IL6I": 0.060,   # 6.0%
    "ABT": 0.045,    # 4.5%
    "JAKi": 0.010,   # 1.0%
    "CS": 0.450,     # 45%
    "NSAID": 0.600   # 60%
}

def generate_medication_records(df_re: pd.DataFrame, df_patients: pd.DataFrame) -> pd.DataFrame:
    """
    各レセプトに対して医薬品レコードを生成
    RA候補患者には論文の使用率に基づいてDMARDsを付与
    """
    records = []
    
    # 患者の属性をマージ
    df_merged = df_re.merge(
        df_patients[['共通キー', 'is_ra_candidate', 'age']],
        on='共通キー',
        how='left'
    )
    
    for _, receipt in df_merged.iterrows():
        line_num = 1
        
        if receipt['is_ra_candidate']:
            # RA候補患者の薬剤処方
            age = receipt['age']
            
            # MTX（年齢による調整：高齢者は使用率低下）
            mtx_rate = DRUG_USAGE_RATES["MTX"]
            if age >= 85:
                mtx_rate *= 0.6
            elif age >= 80:
                mtx_rate *= 0.8
            
            if np.random.random() < mtx_rate:
                records.append(create_medication_record(receipt, "MTX", line_num))
                line_num += 1
            
            # その他のcsDMARDs
            for drug in ["SSZ", "BUC", "TAC", "IGT", "LEF"]:
                if np.random.random() < DRUG_USAGE_RATES[drug]:
                    records.append(create_medication_record(receipt, drug, line_num))
                    line_num += 1
            
            # bDMARDs（年齢によるABT選好の調整）
            bdmard_rate = 0.229  # 22.9%
            if age < 40:
                bdmard_rate *= 2.0  # 若年者は高い
            elif age >= 85:
                bdmard_rate *= 0.6  # 超高齢者は低い
            
            if np.random.random() < bdmard_rate:
                # bDMARDの種類選択（年齢によるABT選好）
                if age >= 70:
                    # 高齢者はABT選好が高い
                    bdmard_type = np.random.choice(["TNFI", "IL6I", "ABT"], p=[0.5, 0.25, 0.25])
                else:
                    bdmard_type = np.random.choice(["TNFI", "IL6I", "ABT"], p=[0.6, 0.25, 0.15])
                
                if bdmard_type == "TNFI":
                    drug = np.random.choice(["IFX", "ETN", "ADA", "GLM", "CZP"])
                elif bdmard_type == "IL6I":
                    drug = np.random.choice(["TCZ", "SAR"])
                else:
                    drug = "ABT"
                
                records.append(create_medication_record(receipt, drug, line_num))
                line_num += 1
            
            # JAK阻害薬（低率）
            if np.random.random() < DRUG_USAGE_RATES["JAKi"]:
                drug = np.random.choice(["TOF", "BAR"])
                records.append(create_medication_record(receipt, drug, line_num))
                line_num += 1
            
            # 経口ステロイド
            if np.random.random() < DRUG_USAGE_RATES["CS"]:
                records.append({
                    "共通キー": receipt["共通キー"],
                    "検索番号": receipt["検索番号"],
                    "データ識別": receipt["データ識別"],
                    "行番号": str(line_num),
                    "枝番号": "0",
                    "レコード識別情報": "IY",
                    "診療識別": "21",
                    "負担区分": "1",
                    "医薬品コード": np.random.choice(CS_CODES),
                    "使用量": str(np.random.randint(1, 30)),
                    "点数": str(np.random.randint(10, 500)),
                    "回数": "1",
                    "drug_category": "CS",
                    "fy": receipt["fy"],
                    "year": receipt["year"],
                    "month": receipt["month"],
                    "prefecture_number": receipt["prefecture_number"]
                })
                line_num += 1
        
        # 全患者に対して一般薬を追加
        if np.random.random() < 0.3:
            records.append({
                "共通キー": receipt["共通キー"],
                "検索番号": receipt["検索番号"],
                "データ識別": receipt["データ識別"],
                "行番号": str(line_num),
                "枝番号": "0",
                "レコード識別情報": "IY",
                "診療識別": "21",
                "負担区分": "1",
                "医薬品コード": f"OTHER{np.random.randint(1000, 9999)}",
                "使用量": str(np.random.randint(1, 30)),
                "点数": str(np.random.randint(10, 500)),
                "回数": "1",
                "drug_category": "OTHER",
                "fy": receipt["fy"],
                "year": receipt["year"],
                "month": receipt["month"],
                "prefecture_number": receipt["prefecture_number"]
            })
    
    return pd.DataFrame(records)

def create_medication_record(receipt, drug_name: str, line_num: int) -> dict:
    """DMARDの医薬品レコードを作成"""
    drug_info = DMARDS_MASTER[drug_name]
    return {
        "共通キー": receipt["共通キー"],
        "検索番号": receipt["検索番号"],
        "データ識別": receipt["データ識別"],
        "行番号": str(line_num),
        "枝番号": "0",
        "レコード識別情報": "IY",
        "診療識別": "21",
        "負担区分": "1",
        "医薬品コード": drug_info["codes"][0],
        "使用量": str(np.random.randint(1, 30)),
        "点数": str(np.random.randint(100, 5000)),
        "回数": "1",
        "drug_category": drug_info["category"],
        "drug_name": drug_name,
        "fy": receipt["fy"],
        "year": receipt["year"],
        "month": receipt["month"],
        "prefecture_number": receipt["prefecture_number"]
    }

df_iy = generate_medication_records(df_re, df_patients)
print(f"生成した医薬品レコード数: {len(df_iy)}")
print(f"\n薬剤カテゴリの分布:")
print(df_iy['drug_category'].value_counts())

## 7. 診療行為データ (SI) の生成

In [ ]:
# RA関連手術コード（簡略化）
RA_SURGERY_CODES = {
    "TJR": {"codes": ["K0821", "K0822"], "name": "人工関節置換術"},
    "ARTHROPLASTY": {"codes": ["K0801", "K0802"], "name": "関節形成術"},
    "SYNOVECTOMY": {"codes": ["K0661", "K0662"], "name": "滑膜切除術"}
}

# 検査コード
IMAGING_CODES = {
    "ULTRASOUND": {"codes": ["D2151"], "name": "関節超音波検査"},
    "BMD": {"codes": ["D2171"], "name": "骨密度測定"}
}

def generate_procedure_records(df_re: pd.DataFrame, df_patients: pd.DataFrame) -> pd.DataFrame:
    """
    各レセプトに対して診療行為レコードを生成
    RA候補患者には一定確率で手術・検査を付与
    """
    records = []
    
    # 患者の属性をマージ
    df_merged = df_re.merge(
        df_patients[['共通キー', 'is_ra_candidate', 'age']],
        on='共通キー',
        how='left'
    )
    
    for _, receipt in df_merged.iterrows():
        line_num = 1
        
        if receipt['is_ra_candidate']:
            age = receipt['age']
            
            # 手術（論文に基づく実施率）
            # 人工関節置換術: 0.93%
            if np.random.random() < 0.0093:
                # 70-84歳で最も多い
                if 70 <= age <= 84:
                    prob_multiplier = 1.5
                else:
                    prob_multiplier = 0.7
                
                if np.random.random() < prob_multiplier:
                    records.append(create_procedure_record(receipt, "TJR", line_num))
                    line_num += 1
            
            # 関節形成術: 0.32%
            if np.random.random() < 0.0032:
                records.append(create_procedure_record(receipt, "ARTHROPLASTY", line_num))
                line_num += 1
            
            # 滑膜切除術: 0.13%（若年者で多い）
            if np.random.random() < 0.0013:
                if age < 70:
                    if np.random.random() < 0.8:  # 若年者でより多い
                        records.append(create_procedure_record(receipt, "SYNOVECTOMY", line_num))
                        line_num += 1
            
            # 関節超音波検査: 約17-19%
            if np.random.random() < 0.18:
                records.append(create_imaging_record(receipt, "ULTRASOUND", line_num))
                line_num += 1
            
            # 骨密度測定: 年齢とともに増加
            bmd_rate = 0.05 + (age / 100) * 0.15  # 年齢に応じて5-20%
            if np.random.random() < bmd_rate:
                records.append(create_imaging_record(receipt, "BMD", line_num))
                line_num += 1
        
        # 基本的な診療行為（全患者）
        records.append({
            "共通キー": receipt["共通キー"],
            "検索番号": receipt["検索番号"],
            "データ識別": receipt["データ識別"],
            "行番号": str(line_num),
            "枝番号": "0",
            "レコード識別情報": "SI",
            "診療識別": "11",
            "負担区分": "1",
            "診療行為コード": "111000110",  # 初診料等
            "数量データ": "1",
            "点数": str(np.random.randint(100, 300)),
            "回数": "1",
            "procedure_type": "VISIT",
            "fy": receipt["fy"],
            "year": receipt["year"],
            "month": receipt["month"],
            "prefecture_number": receipt["prefecture_number"]
        })
    
    return pd.DataFrame(records)

def create_procedure_record(receipt, surgery_type: str, line_num: int) -> dict:
    """手術の診療行為レコードを作成"""
    surgery_info = RA_SURGERY_CODES[surgery_type]
    return {
        "共通キー": receipt["共通キー"],
        "検索番号": receipt["検索番号"],
        "データ識別": receipt["データ識別"],
        "行番号": str(line_num),
        "枝番号": "0",
        "レコード識別情報": "SI",
        "診療識別": "50",  # 手術
        "負担区分": "1",
        "診療行為コード": surgery_info["codes"][0],
        "数量データ": "1",
        "点数": str(np.random.randint(10000, 50000)),
        "回数": "1",
        "procedure_type": surgery_type,
        "procedure_name": surgery_info["name"],
        "fy": receipt["fy"],
        "year": receipt["year"],
        "month": receipt["month"],
        "prefecture_number": receipt["prefecture_number"]
    }

def create_imaging_record(receipt, imaging_type: str, line_num: int) -> dict:
    """検査の診療行為レコードを作成"""
    imaging_info = IMAGING_CODES[imaging_type]
    return {
        "共通キー": receipt["共通キー"],
        "検索番号": receipt["検索番号"],
        "データ識別": receipt["データ識別"],
        "行番号": str(line_num),
        "枝番号": "0",
        "レコード識別情報": "SI",
        "診療識別": "60",  # 検査
        "負担区分": "1",
        "診療行為コード": imaging_info["codes"][0],
        "数量データ": "1",
        "点数": str(np.random.randint(100, 1000)),
        "回数": "1",
        "procedure_type": imaging_type,
        "procedure_name": imaging_info["name"],
        "fy": receipt["fy"],
        "year": receipt["year"],
        "month": receipt["month"],
        "prefecture_number": receipt["prefecture_number"]
    }

df_si = generate_procedure_records(df_re, df_patients)
print(f"生成した診療行為レコード数: {len(df_si)}")
print(f"\n診療行為タイプの分布:")
print(df_si['procedure_type'].value_counts())

## 8. 保険者データ (HO) の生成

In [ ]:
def generate_insurer_records(df_re: pd.DataFrame) -> pd.DataFrame:
    """
    各レセプトに対して保険者レコードを生成
    """
    records = []
    
    for _, receipt in df_re.iterrows():
        records.append({
            "共通キー": receipt["共通キー"],
            "検索番号": receipt["検索番号"],
            "データ識別": receipt["データ識別"],
            "行番号": "1",
            "枝番号": "0",
            "レコード識別情報": "HO",
            "保険者番号": f"{np.random.randint(10000000, 99999999)}",
            "被保険者証等記号": f"SYM{np.random.randint(1000, 9999)}",
            "被保険者証等番号": f"{np.random.randint(100000, 999999)}",
            "診療実日数": str(np.random.randint(1, 15)),
            "合計点数": str(np.random.randint(100, 10000)),
            "fy": receipt["fy"],
            "year": receipt["year"],
            "month": receipt["month"],
            "prefecture_number": receipt["prefecture_number"]
        })
    
    return pd.DataFrame(records)

df_ho = generate_insurer_records(df_re)
print(f"生成した保険者レコード数: {len(df_ho)}")

## 9. データの保存

In [ ]:
# Bronzeデータとして保存
df_patients.to_parquet(f"{BRONZE_DIR}/patients.parquet", index=False)
df_re.to_parquet(f"{BRONZE_DIR}/re_receipt.parquet", index=False)
df_sy.to_parquet(f"{BRONZE_DIR}/sy_disease.parquet", index=False)
df_iy.to_parquet(f"{BRONZE_DIR}/iy_medication.parquet", index=False)
df_si.to_parquet(f"{BRONZE_DIR}/si_procedure.parquet", index=False)
df_ho.to_parquet(f"{BRONZE_DIR}/ho_insurer.parquet", index=False)

print("Bronzeデータを保存しました:")
print(f"  - patients.parquet: {len(df_patients)} records")
print(f"  - re_receipt.parquet: {len(df_re)} records")
print(f"  - sy_disease.parquet: {len(df_sy)} records")
print(f"  - iy_medication.parquet: {len(df_iy)} records")
print(f"  - si_procedure.parquet: {len(df_si)} records")
print(f"  - ho_insurer.parquet: {len(df_ho)} records")

## 10. データ品質チェック

In [ ]:
print("=" * 60)
print("Bronzeデータ品質サマリー")
print("=" * 60)

# RA候補患者の統計
ra_candidates = df_patients[df_patients['is_ra_candidate']]
print(f"\n【患者統計】")
print(f"総患者数: {len(df_patients)}")
print(f"RA候補患者数: {len(ra_candidates)}")
print(f"RA候補患者の女性比率: {(ra_candidates['sex'] == '2').mean():.1%}")

# 年齢分布
print(f"\n【RA候補患者の年齢分布】")
age_dist = ra_candidates['age_group'].value_counts()
for group in [g[0] for g in AGE_GROUPS]:
    count = age_dist.get(group, 0)
    pct = count / len(ra_candidates) * 100
    paper_pct = AGE_GROUP_RA_DISTRIBUTION.get(group, 0)
    print(f"  {group}: {count} ({pct:.1f}%) [論文: {paper_pct}%]")

# ICD-10コードの分布
ra_codes = df_sy[df_sy['ICD10コード'].isin(RA_ICD10_CODES)]
print(f"\n【RA関連ICD-10コードを持つレコード数】")
print(f"  {len(ra_codes)} records")

# DMARDs処方の分布
dmard_records = df_iy[df_iy['drug_category'].isin(['csDMARD', 'TNFI', 'IL6I', 'ABT', 'JAKi'])]
print(f"\n【DMARDs処方レコード数】")
print(df_iy[df_iy['drug_category'] != 'OTHER']['drug_category'].value_counts())

print("\n" + "=" * 60)
print("Bronze Layer 生成完了")
print("=" * 60)